In [1]:
                                        ## Project Nr. 1
                                        ### ETL REVIEWS

ETL_REVIEWS

In [2]:

##  Import Libraries
import ast
import pandas as pd
import fastparquet
from textblob import TextBlob

In [3]:
data_list = []

In [4]:
#JSON file path
file_path = (r'..\Extras\australian_user_reviews.json')

#Open the file and process each line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Use ast.literal_eval to convert the line to a dictionary
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Online Error: {line}")
            continue

In [5]:
#Create a DataFrame from the list of dictionaries
data_it = pd.DataFrame(data_list)

In [6]:
data_it1 = data_it.explode(['reviews'])
data_it2 = pd.json_normalize(data_it1['reviews']).set_index(data_it1['reviews'].index)
data_it3= pd.concat([data_it2, data_it1],axis=1)


In [7]:
data_it3 

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...,...,...,...,...
25797,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [8]:
#Import the textblob library to perform a sentiment analysis on Reviews


In [9]:
#Define the function to determine if the user review is negative, neutral and positive. 
#Catching 0,1,2 respectively in a new column 'sentiment_analysis'

def sentiment_analysis(review):
    if isinstance(review, list) and len(review) > 0:
        text = review[0].get('review', '') # Get the review text
        sentiment = TextBlob(text).sentiment.polarity

        if sentiment < -0.2:
            return 0    # Bad
        elif sentiment >= -0.2 and sentiment <= 0.2:
            return 1    # Neutral
        else:
            return 2    # Positive
    else:
        return 1        # Default value for missing reviews

In [10]:
# Apply the function to the 'reviews' column and create the new 'sentiment_analysis' column
data_it ['sentiment_analysis'] = data_it ['reviews'].apply(sentiment_analysis)
data_it 

,user_id,user_url,reviews,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",1
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",1
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",1
...,...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la...",2
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l...",1
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',...",1
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l...",2


In [11]:
# data_unnested is a list used to store the unnested review data and other details.
data_unnested = []

# Iterate over the rows of the DataFrame data_it, where index is the row index and row is an object containing the values of each column in that row.
for index, row in data_it.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    sentiment_analysis_value = row['sentiment_analysis']  # Get the value of sentiment_analysis
    reviews = row['reviews']

    # The following for loop iterates over each review in the reviews list for that row.
    for i in reviews:
        # Create a dictionary that stores all relevant values from the current row, including unnested review values
        # and the sentiment analysis result calculated using the sentiment_analysis function.
        new_row = {
            'user_id': user_id,
            'user_url': user_url,
            'reviews': reviews,
            'sentiment_analysis': sentiment_analysis_value,
            'funny': i.get('funny', ''),
            'posted': i.get('posted', ''),
            'last_edited': i.get('last_edited', ''),
            'item_id': i.get('item_id', ''),
            'helpful': i.get('helpful', ''),
            'recommend': i.get('recommend', False),
            'review': i.get('review', '')
        }
        # Add the new_row dictionary to the data_unnested list.
        data_unnested.append(new_row)

# Create a DataFrame from the data_unnested list
data_it_unnested = pd.DataFrame(data_unnested)

In [12]:
# Remove unnecessary columns from the DataFrame
data_it_unnested.drop(['reviews', 'helpful', 'review', 'last_edited', 'funny'], axis=1, inplace=True)
data_it_unnested

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted November 5, 2011.",1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted July 15, 2011.",22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"Posted April 21, 2011.",43110,True
3,js41637,http://steamcommunity.com/id/js41637,1,"Posted June 24, 2014.",251610,True
4,js41637,http://steamcommunity.com/id/js41637,1,"Posted September 8, 2013.",227300,True
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,Posted July 10.,70,True
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,Posted July 8.,362890,True
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,Posted July 3.,273110,True
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,Posted July 20.,730,True


In [13]:
# Change the date format

from dateutil import parser

# Function to parse the date and handle errors
def parse_date(date_str):
    try:
        return parser.parse(date_str.replace("Posted ", ""), fuzzy=True)
    except ValueError:
        return None

# Apply the date parsing function and replace rows with incorrect dates with NaN
data_it_unnested['posted'] = data_it_unnested['posted'].apply(parse_date)

# Remove rows with incorrect dates (NaN in the "posted" column)
data_it_unnested = data_it_unnested.dropna(subset=['posted'])

# Now, the "posted" column contains dates in the desired format, and rows with incorrect dates have been removed
data_it_unnested

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-11-05,1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-07-15,22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-04-21,43110,True
3,js41637,http://steamcommunity.com/id/js41637,1,2014-06-24,251610,True
4,js41637,http://steamcommunity.com/id/js41637,1,2013-09-08,227300,True
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-10,70,True
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-08,362890,True
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-03,273110,True
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-20,730,True


In [14]:
# Delete the duplicate rows based on all columns and keep only the last occurrence of duplicates
data_it_unnested.drop_duplicates(keep='last', inplace=True)

This dataset contains 3 columns and 25,458 rows, with no missing values. The columns are as follows:

user_id: A unique identifier for the user.
user_url: The user’s profile URL on Streamcommunity.

reviews: A list of dictionaries. For each user, there is one or more dictionaries representing reviews. Each dictionary contains the following information:
    funny: Indicates whether someone added a funny emoji to the review.
    posted: The date when the review was posted in the format “Posted April 21, 2011.”
    last_edited: The date of the last edit.
    item_id: The unique identifier for the item (i.e., the game).
    helpful: A statistic where other users indicate whether the information was helpful.
    recommend: A boolean indicating whether the user recommends the game.
    review: A string sentence with comments about the game.

In [15]:
data_it_unnested['item_id'] = data_it_unnested['item_id'].astype(str)

In [16]:
# Save the DataFrame to the CSV file
data_it_unnested.to_csv('./CleanDatasets/reviews.csv', index=False)
print("CSV file saved successfully!")

CSV file saved successfully!


In [17]:
#Read DF and rename

df_reviews = pd.read_csv('./CleanDatasets/reviews.csv')
df_reviews


,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-11-05,1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-07-15,22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,2011-04-21,43110,True
3,js41637,http://steamcommunity.com/id/js41637,1,2014-06-24,251610,True
4,js41637,http://steamcommunity.com/id/js41637,1,2013-09-08,227300,True
...,...,...,...,...,...,...
58426,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-10,70,True
58427,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2,2024-07-08,362890,True
58428,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-03,273110,True
58429,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1,2024-07-20,730,True


In [18]:
def ddbb_verification(df_reviews):
    mi_dict = {"name": [], "tipe": [], "no_nulls_%": [], "nulls_%": [], "nulls": []}

    for column in df_reviews.columns:
        porcentaje_no_nulls = (df_reviews[column].count() / len(df_reviews)) * 100
        mi_dict["name"].append(column)
        mi_dict["tipe"].append(df_reviews[column].apply(type).unique())
        mi_dict["no_nulls_%"].append(round(porcentaje_no_nulls, 2))
        mi_dict["nulls_%"].append(round(100 - porcentaje_no_nulls, 2))
        mi_dict["nulls"].append(df_reviews[column].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info

# Call the function
df_reviews_info = ddbb_verification(df_reviews)
print(df_reviews_info)


                 name              tipe  no_nulls_%  nulls_%  nulls
0             user_id   [<class 'str'>]       100.0      0.0      0
1            user_url   [<class 'str'>]       100.0      0.0      0
2  sentiment_analysis   [<class 'int'>]       100.0      0.0      0
3              posted   [<class 'str'>]       100.0      0.0      0
4             item_id   [<class 'int'>]       100.0      0.0      0
5           recommend  [<class 'bool'>]       100.0      0.0      0
